In [4]:
import open3d as o3d
import numpy as np
import os
import glob
import pandas as pd
import pymeshlab as pml
import time
import matplotlib as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
from scipy.stats import wasserstein_distance
from emd import get_emd

In [6]:
wasserstein_distance([0, 1, 3], [5, 6, 8])

5.0

In [144]:
df = pd.read_pickle("./normalized_features_final.pkl")
df.reset_index(drop=True, inplace=True)

In [145]:
df

,class,path,V,S,c,D,R,E,C,A3,D1,D2,D3,D4
0,AircraftBuoyant,./features/AircraftBuoyant\m1337,4.100846,0.214806,-0.021151,-0.431171,1.524371,4.945361,1.790998,"[0.000719, 0.002245, 0.003637, 0.004885, 0.006...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.00048, 0.00051, 0.00104, 0.00095, 0.00...","[1e-06, 5e-06, 1.9e-05, 5.4e-05, 0.000111, 0.0...","[1e-05, 3.7e-05, 0.000109, 0.000236, 0.000379,..."
1,AircraftBuoyant,./features/AircraftBuoyant\m1338,4.728118,0.540607,-0.021151,-0.117928,1.525120,3.309235,1.818866,"[0.000883, 0.002546, 0.003664, 0.004831, 0.006...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9e-05, 0.00037, 0.00058, 0.00118, 0.00128, 0....","[1e-06, 1e-05, 4.8e-05, 0.000111, 0.000193, 0....","[5e-06, 4.4e-05, 0.000137, 0.000293, 0.000453,..."
2,AircraftBuoyant,./features/AircraftBuoyant\m1339,0.087864,0.155798,-0.021151,-0.690660,0.052891,0.829243,0.072976,"[0.002071, 0.005089, 0.007236, 0.009431, 0.010...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8.000080000800008e-05, 0.0007100071000710007,...","[2e-06, 1.6e-05, 5.8e-05, 0.000134, 0.000256, ...","[1.1e-05, 8.2e-05, 0.000193, 0.000388, 0.00063..."
3,AircraftBuoyant,./features/AircraftBuoyant\m1340,0.219291,-0.646455,-0.021151,-0.689441,0.445397,-0.294062,0.667100,"[0.006885, 0.012406, 0.016479, 0.019964, 0.023...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.00081, 0.0025, 0.00407, 0.0057, 0.00623, 0....","[1e-06, 7e-05, 0.000194, 0.000365, 0.000693, 0...","[7.000007000007e-06, 0.000101000101000101, 0.0..."
4,AircraftBuoyant,./features/AircraftBuoyant\m1341,-0.458655,-0.605102,-0.021151,-0.691012,-1.082039,-0.191202,-1.437485,"[0.004617, 0.010763, 0.015059, 0.018871, 0.021...","[0.0, 0.0028, 0.0054, 0.0038, 0.0047, 0.0062, ...","[0.0007400074000740007, 0.0014100141001410014,...","[3e-06, 4e-05, 9.9e-05, 0.000255, 0.000431, 0....","[2e-05, 9.6e-05, 0.000282, 0.000558, 0.000878,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,WheelChair,./features/WheelChair\D00687,-0.257985,-0.136291,-0.021151,-0.318480,-0.355524,0.236439,-0.413690,"[0.010268, 0.014988, 0.014769, 0.013699, 0.012...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.00252, 0.00417, 0.00609, 0.00682, 0.00829, ...","[5.7e-05, 0.000282, 0.000791, 0.001575, 0.0022...","[6.6e-05, 0.000352, 0.000985, 0.001733, 0.0026..."
2471,WheelChair,./features/WheelChair\D00720,0.415856,-0.265226,-0.021151,-0.563298,0.443687,0.331357,0.558552,"[0.003035003035003035, 0.006674006674006674, 0...","[0.0, 0.0002, 0.0, 0.0, 0.0, 0.0006, 0.0014, 0...","[0.00066, 0.00121, 0.00223, 0.00334, 0.00465, ...","[1.8e-05, 5.2e-05, 0.000175, 0.000367, 0.00055...","[1.9e-05, 0.000139, 0.000417, 0.000785, 0.0012..."
2472,WheelChair,./features/WheelChair\D00782,0.330954,-0.270341,-0.021151,-0.133080,0.251680,0.460616,0.345153,"[0.003472, 0.00729, 0.009331, 0.011377, 0.0129...","[0.0, 0.0002, 0.0, 0.0001, 0.0003, 0.0, 0.0, 0...","[0.00034, 0.00147, 0.00244, 0.00295, 0.00331, ...","[1.4e-05, 4.6e-05, 0.000168, 0.000317, 0.0006,...","[3.9e-05, 0.000155, 0.000407, 0.000801, 0.0012..."
2473,WheelChair,./features/WheelChair\D00821,0.315740,-0.136219,-0.021151,-0.316797,0.369517,0.233050,0.444380,"[0.010317, 0.014894, 0.015066, 0.01355, 0.0130...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.00252, 0.00428, 0.00618, 0.00689, 0.00781, ...","[4.6e-05, 0.000297, 0.000783, 0.001427, 0.0021...","[6.5e-05, 0.000346, 0.000939, 0.00174, 0.00262..."


In [8]:
def w_dist(a,b):
    return wasserstein_distance(a,b)*1000

In [9]:
valdf = df[['A3', 'D1', 'D2', 'D3', 'D4']]

In [10]:
newobj = df.iloc()[1300][['A3', 'D1', 'D2', 'D3', 'D4']]
newobj

A3    [0.005445, 0.012587, 0.016992, 0.020727, 0.023...
D1    [0.0, 0.0006, 0.0032, 0.0038, 0.0045, 0.0036, ...
D2    [0.00047, 0.0017, 0.0029, 0.00443, 0.00618, 0....
D3    [2.4e-05, 0.000276, 0.000935, 0.002035, 0.0036...
D4    [5e-05, 0.000375, 0.00089, 0.001752, 0.002815,...
Name: 1300, dtype: object

In [69]:
w1=1;w2=1;w3=1;w4=1;w5=1
weights = np.array([w1,w2,w3,w4,w5])

In [70]:
weighted_sums = np.array([])
for index, row in valdf.iterrows():
        # print(valdf.columns[0])
        values = np.array([])
        for col in valdf.columns:
            dist = w_dist(newobj[col],row[col])
            # dist = get_emd(newobj[col],row[col])

            values = np.append(values, dist)
        weighted_sum = np.dot(values, weights)
        weighted_sums = np.append(weighted_sums, weighted_sum)
        # print(weighted_sum)

In [71]:
lowest_indices = np.argsort(weighted_sums)[:10]

lowest_values = weighted_sums[lowest_indices]

print("5 Lowest Values:", lowest_values)
print("Indices of the 5 Lowest Values:", lowest_indices)


5 Lowest Values: [0.         2.11851115 2.97657638 3.08118894 3.20097564 3.38957091
 3.42135312 3.63629361 3.72862602 3.84290286]
Indices of the 5 Lowest Values: [1300 1318 1309 1351 1364 1335  918 1310 1284 1361]


In [72]:
df.iloc[lowest_indices]["class"]

1           Jet
1           Jet
1           Jet
1           Jet
1           Jet
1           Jet
1    Helicopter
1           Jet
1           Jet
1           Jet
Name: class, dtype: object

In [ ]:
df[df["class"]=="Vase"]

In [10]:
df[df.apply(lambda x: x.isnull().any(), axis=1)]


,class,path,V,S,c,D,R,E,C,A3,D1,D2,D3,D4


In [11]:
def has_nan_in_list(row):
    return any(pd.isna(x) for x in row['D1'])
df_has_nan = df.apply(has_nan_in_list, axis=1)
df[df_has_nan]

,class,path,V,S,c,D,R,E,C,A3,D1,D2,D3,D4


In [65]:
import sys
sys.path.append('../preprocessing')
from viz import viz_mesh

In [85]:
viz_mesh("../resampledPML//Chess\m1591.obj"	)

In [75]:
def give_random_triplets(input=range(10), samples=10):
    return np.random.choice(input, size=(samples, 3), replace=True)

random_triplets = give_random_triplets(input=range(1000), samples=1000000)
random_triplets

array([[756, 820, 553],
       [ 86, 287,  27],
       [226, 402, 564],
       ...,
       [355, 783, 267],
       [586,  81, 968],
       [387, 647, 719]])

In [76]:
equal_indices = np.where((random_triplets[:, 0] == random_triplets[:, 1]) | (random_triplets[:, 1] == random_triplets[:, 2]) | (random_triplets[:, 0] == random_triplets[:, 2]))
equal_indices[0]

array([   454,    519,    583, ..., 999291, 999627, 999783], dtype=int64)

In [77]:
random_triplets[equal_indices] = give_random_triplets(samples=random_triplets[equal_indices].shape[0] )


In [194]:
random_triplets = give_random_triplets(input=range(7000), samples=1000000)
equal_indices = np.where((random_triplets[:, 0] == random_triplets[:, 1]) | (random_triplets[:, 1] == random_triplets[:, 2]) | (random_triplets[:, 0] == random_triplets[:, 2]))
while equal_indices[0].size > 0:
    random_triplets[equal_indices] = give_random_triplets(samples=random_triplets[equal_indices].shape[0] )
    equal_indices = np.where((random_triplets[:, 0] == random_triplets[:, 1]) | (random_triplets[:, 1] == random_triplets[:, 2]) | (random_triplets[:, 0] == random_triplets[:, 2]))

random_triplets


58.2 ms ± 4.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Accuracy measures

In [196]:
def get_top_k_results(newobj, k=10):
    w1=1;w2=1;w3=1;w4=4;w5=3
    weights = np.array([w1,w2,w3,w4,w5])
    weighted_sums = np.array([])
    for index, row in valdf.iterrows():
        # print(valdf.columns[0])
        values = np.array([])
        for col in valdf.columns:
            dist = w_dist(newobj[col],row[col])
            # dist = get_emd(newobj[col],row[col])

            values = np.append(values, dist)
        weighted_sum = np.dot(values, weights)
        weighted_sums = np.append(weighted_sums, weighted_sum)
        # print(weighted_sum)
        lowest_indices = np.argsort(weighted_sums)[:k+1]

        lowest_values = weighted_sums[lowest_indices]

    print("5 Lowest Values:", lowest_values)
    print("Indices of the 5 Lowest Values:", lowest_indices)
    return lowest_indices, lowest_values


In [15]:
df.columns[9:]

Index(['A3', 'D1', 'D2', 'D3', 'D4'], dtype='object')

In [147]:
def get_top_k_results_allfeatures(newobj, k=10, verbose=0):
    w0=5;w1=1;w2=1;w3=1;w4=1;w5=1
    weights = np.array([w0,w1,w2,w3,w4,w5])
    weighted_sums = np.array([])
    for index, row in df.iterrows():
        # print(valdf.columns[0])
        values = np.array([])
        dist = euclidean_distance(newobj,row)
        values = np.append(values, dist)
        # dist2 = cosine_distance(newobj,row)
        # values = np.append(values, dist2)
            
        for col in df.columns[9:]:
            dist3 = w_dist(newobj[col],row[col])
            # dist = get_emd(newobj[col],row[col])

            values = np.append(values, dist3)
        weighted_sum = np.dot(values, weights)
        weighted_sums = np.append(weighted_sums, weighted_sum)
        # print(weighted_sum)
        lowest_indices = np.argsort(weighted_sums)[:k+1]

        lowest_values = weighted_sums[lowest_indices]
    if verbose==1:
        print("5 Lowest Values:", lowest_values)
        print("Indices of the 5 Lowest Values:", lowest_indices)
    return lowest_indices, lowest_values


In [94]:

newobj = df.iloc()[130]#[['A3', 'D1', 'D2', 'D3', 'D4']]
newobj

class                                              Biplane
path                             ./features/Biplane\D00410
V                                                -0.073661
S                                                -0.498022
c                                                -0.021151
D                                                -0.654631
R                                                -0.269601
E                                                -0.535418
C                                                -0.374068
A3       [0.00518, 0.011636, 0.014856, 0.018058, 0.0208...
D1       [0.0001, 0.0001, 0.0003, 0.0008, 0.0021, 0.001...
D2       [0.00052, 0.00179, 0.0035, 0.00489, 0.00652, 0...
D3       [1.7e-05, 0.0001, 0.000383, 0.000934, 0.001647...
D4       [0.000216, 0.00022, 0.000556, 0.001181, 0.0018...
Name: 130, dtype: object

In [95]:
lowest_indices, lowest_values = get_top_k_results_allfeatures(newobj=newobj)

5 Lowest Values: [0.         3.20105525 3.23270059 3.2584804  3.38854713 3.51740614
 3.59079407 3.64372214 3.67745789 3.75281822 3.91014752]
Indices of the 5 Lowest Values: [ 130 1216 1224  994 1131 2271  962  146  645 1201 1002]


In [96]:
df.iloc[lowest_indices]["class"]

130        Biplane
1216        Insect
1224        Insect
994          House
1131      Humanoid
2271          Tool
962     Helicopter
146        Biplane
645      DeskPhone
1201      Humanoid
1002         House
Name: class, dtype: object

In [99]:
import pandas as pd
import random

# Create an empty dictionary to store the random indices
random_indices_dict = {}
indices_arr = np.array([],dtype=int)
# Group the DataFrame by the "class" column
grouped = df.groupby('class')

# Iterate over each group and select two different random indices for each
for name, group in grouped:
    indices = group.index.tolist()
    newvals = random.sample(indices, min(2, len(indices)))
    random_indices_dict[name] = newvals
    indices_arr = np.append(indices_arr,newvals)

# Display the random indices for each distinct class
print(random_indices_dict)
print(indices_arr)


{'AircraftBuoyant': [3, 7], 'Apartment': [17, 21], 'AquaticAnimal': [40, 65], 'Bed': [78, 81], 'Bicycle': [104, 99], 'Biplane': [160, 147], 'Bird': [208, 193], 'Bookset': [217, 218], 'Bottle': [242, 258], 'BuildingNonResidential': [271, 276], 'Bus': [308, 318], 'Car': [387, 359], 'Cellphone': [420, 417], 'Chess': [469, 442], 'City': [484, 496], 'ClassicPiano': [514, 508], 'Computer': [548, 540], 'ComputerKeyboard': [563, 557], 'Cup': [586, 587], 'DeskLamp': [609, 632], 'DeskPhone': [639, 636], 'Door': [663, 673], 'Drum': [698, 702], 'Fish': [717, 735], 'FloorLamp': [744, 754], 'Glasses': [773, 775], 'Guitar': [799, 796], 'Gun': [847, 842], 'Hand': [864, 863], 'Hat': [912, 899], 'Helicopter': [943, 935], 'House': [971, 991], 'HumanHead': [1040, 1019], 'Humanoid': [1188, 1176], 'Insect': [1230, 1222], 'Jet': [1364, 1290], 'Knife': [1411, 1422], 'MilitaryVehicle': [1452, 1446], 'Monitor': [1464, 1474], 'Monoplane': [1481, 1496], 'Motorcycle': [1522, 1500], 'Mug': [1549, 1527], 'MultiSeat'

In [195]:
get_top_k_results(newobj, k = 10)

5 Lowest Values: [0.         2.6673     3.0845001  3.16317584 3.19272594 3.20122
 3.26587334 3.29074614 3.32684614 3.39054438 3.39365327]
Indices of the 5 Lowest Values: [2461 2458 2334 1982 1984  876 1974  897 1633 1555 1590]


(array([2461, 2458, 2334, 1982, 1984,  876, 1974,  897, 1633, 1555, 1590],
       dtype=int64),
 array([0.        , 2.6673    , 3.0845001 , 3.16317584, 3.19272594,
        3.20122   , 3.26587334, 3.29074614, 3.32684614, 3.39054438,
        3.39365327]))

In [149]:
from tqdm import tqdm
totalcorrect = 0
total = 0
for i in tqdm(indices_arr):
    newobj = df.iloc()[i]
    # lowest_indices, lowest_values = get_top_k_results(newobj, k = 10)
    lowest_indices, lowest_values = get_top_k_results_allfeatures(newobj, k = 10, verbose=0)

    y = df.iloc[lowest_indices[0]]["class"]
    correct = 0
    for ind in lowest_indices[1:]:
        if df.iloc[ind]["class"] == y:
            correct += 1 
    totalcorrect += correct
    total += len(lowest_indices[1:])

100%|██████████| 138/138 [07:36<00:00,  3.31s/it]


In [150]:
totalcorrect, total, totalcorrect/ total # with new df

(335, 1380, 0.2427536231884058)

In [105]:
totalcorrect, total, totalcorrect/ total # with euclid

(307, 1380, 0.22246376811594204)

In [102]:
totalcorrect, total, totalcorrect/ total # with cosine

(287, 1380, 0.20797101449275363)

In [199]:
totalcorrect, total, totalcorrect/ total

(209, 1380, 0.15144927536231884)

In [192]:
totalcorrect, total, totalcorrect/ total

(238, 1380, 0.17246376811594202)

In [188]:
df.iloc[lowest_indices]

,class,path,V,S,c,D,R,E,C,A3,D1,D2,D3,D4
100,Bicycle,./features/Bicycle\D00462,0.015860,-0.558199,-0.021151,-0.530023,-0.014967,-0.675910,-0.021856,"[0.013414, 0.021464, 0.02267, 0.022876, 0.0231...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00175, 0.00451, 0.00605, 0.00585, 0.00708, ...","[0.00012, 0.000591, 0.001199, 0.001893, 0.0027...","[0.000103, 0.000612, 0.001531, 0.002789, 0.004..."
102,Bicycle,./features/Bicycle\D00536,-0.003958,-0.594845,-0.021151,-0.509049,-0.024217,-0.458325,-0.038430,"[0.011727, 0.017246, 0.021203, 0.021615, 0.021...","[0.0, 0.0001, 0.0001, 0.0001, 0.0002, 0.0001, ...","[0.00227, 0.0065, 0.00635, 0.00573, 0.00592, 0...","[0.00015, 0.000766, 0.001386, 0.002328, 0.0032...","[0.000144, 0.000953, 0.002239, 0.003976, 0.006..."
1465,Monitor,./features/Monitor\D00391,-0.179488,1.204441,-0.021151,0.852796,-0.389738,-0.702152,-0.545376,"[0.009992, 0.01945, 0.021982, 0.022273, 0.0224...","[0.0, 0.001, 0.0011, 0.0019, 0.001, 0.0011, 0....","[0.00073, 0.002, 0.00271, 0.0033, 0.00487, 0.0...","[3.2e-05, 0.000166, 0.000468, 0.000992, 0.0016...","[0.000744, 0.000818, 0.001055, 0.001956, 0.002..."
108,Bicycle,./features/Bicycle\D00750,0.100926,-0.272764,-0.021151,-0.372126,0.098439,-0.604313,0.151024,"[0.009781, 0.019754, 0.023327, 0.023564, 0.023...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00054, 0.00234, 0.00401, 0.00534, 0.00647, ...","[1.4e-05, 0.000117, 0.000433, 0.001004, 0.0017...","[6.6e-05, 0.000416, 0.001103, 0.001944, 0.0033..."
114,Bicycle,./features/Bicycle\D00974,-0.024465,-0.548177,-0.021151,-0.498631,-0.117300,-0.688648,-0.187253,"[0.011226, 0.021829, 0.026766, 0.02798, 0.0269...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00158, 0.00473, 0.00715, 0.00809, 0.00925, ...","[3.7e-05, 0.000395, 0.000988, 0.001959, 0.0030...","[7.6e-05, 0.000494, 0.001341, 0.002521, 0.0039..."
96,Bicycle,./features/Bicycle\D00033,0.011966,-0.672358,-0.021151,-0.687026,-0.021327,-0.704064,-0.034429,"[0.012993, 0.022425, 0.024989, 0.025044, 0.026...","[0.0, 0.0001, 0.0001, 0.0004, 0.0002, 0.0003, ...","[0.00169, 0.00331, 0.00459, 0.00462, 0.00762, ...","[8.7e-05, 0.00057, 0.001067, 0.001509, 0.00225...","[0.000108, 0.000687, 0.001729, 0.00331, 0.0050..."
115,Bicycle,./features/Bicycle\m1472,0.001595,-0.627271,-0.021151,-0.647436,-0.019638,-0.642163,-0.038314,"[0.011378, 0.022022, 0.024566, 0.024305, 0.023...","[0.0, 0.0001, 0.0001, 0.0001, 0.0012, 0.0005, ...","[0.00138, 0.00399, 0.00619, 0.00623, 0.00645, ...","[2.6e-05, 0.000247, 0.000791, 0.00152, 0.00252...","[0.000117, 0.000755, 0.001949, 0.003755, 0.005..."
106,Bicycle,./features/Bicycle\D00621,0.045538,-0.656544,-0.021151,-0.634887,0.044704,-0.723747,0.060357,"[0.016849, 0.026814, 0.029573, 0.028311, 0.027...","[0.0, 0.0001, 0.0001, 0.0001, 0.0005, 0.0009, ...","[0.00165, 0.00388, 0.00539, 0.00654, 0.00796, ...","[0.000116, 0.000666, 0.001466, 0.00253, 0.0038...","[0.000143, 0.001057, 0.002642, 0.004558, 0.007..."
104,Bicycle,./features/Bicycle\D00558,-0.014542,-0.538586,-0.021151,-0.656887,-0.077964,-0.676072,-0.103970,"[0.009977, 0.019977, 0.023832, 0.024862, 0.024...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00107, 0.00332, 0.00505, 0.006, 0.00729, 0....","[3.2e-05, 0.000223, 0.00072, 0.001249, 0.00209...","[7e-05, 0.00045, 0.001221, 0.002446, 0.00398, ..."
99,Bicycle,./features/Bicycle\D00078,0.067563,-0.551459,-0.021151,-0.280044,0.052743,-0.578388,0.088031,"[0.008261, 0.015829, 0.019215, 0.021177, 0.021...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00165, 0.00433, 0.00711, 0.00694, 0.00803, ...","[4.8e-05, 0.000358, 0.000952, 0.001911, 0.0029...","[7.2e-05, 0.000499, 0.001296, 0.002559, 0.0040..."


In [11]:
df2 = df

In [21]:
min(df[['A3', 'D1', 'D2', 'D3', 'D4']].applymap(len).min())

12

In [148]:
list_lengths = df[['A3', 'D1', 'D2', 'D3', 'D4']].applymap(len)

# Find the row indices with the maximum and minimum list lengths
row_with_max_length = list_lengths.sum(axis=1).idxmax()
row_with_min_length = list_lengths.sum(axis=1).idxmin()

# Get the corresponding rows from the original DataFrame
row_with_max_length_data = df.iloc[row_with_max_length]
row_with_min_length_data = df.iloc[row_with_min_length]

# Find the columns with the maximum and minimum list lengths
max_length_columns = list_lengths.loc[row_with_max_length].idxmax()
min_length_columns = list_lengths.loc[row_with_min_length].idxmin()

# Get the respective lengths of the maximum and minimum list lengths
max_length = list_lengths.loc[row_with_max_length, max_length_columns]
min_length = list_lengths.loc[row_with_min_length, min_length_columns]

print("Row with the highest list length (Row Index:", row_with_max_length, ")")
print(row_with_max_length_data)
print("Columns with the highest list length:", max_length_columns)
print("Length:", max_length)

print("\nRow with the lowest list length (Row Index:", row_with_min_length, ")")
print(row_with_min_length_data)
print("Columns with the lowest list length:", min_length_columns)
print("Length:", min_length)

Row with the highest list length (Row Index: 0 )
class                                      AircraftBuoyant
path                      ./features/AircraftBuoyant\m1337
V                                                 4.100846
S                                                 0.214806
c                                                -0.021151
D                                                -0.431171
R                                                 1.524371
E                                                 4.945361
C                                                 1.790998
A3       [0.000719, 0.002245, 0.003637, 0.004885, 0.006...
D1       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
D2       [0.0, 0.00048, 0.00051, 0.00104, 0.00095, 0.00...
D3       [1e-06, 5e-06, 1.9e-05, 5.4e-05, 0.000111, 0.0...
D4       [1e-05, 3.7e-05, 0.000109, 0.000236, 0.000379,...
Name: 0, dtype: object
Columns with the highest list length: A3
Length: 100

Row with the lowest list length (Row Index: 0 )


In [37]:
df[df["path"] == "./features/Car\m1510"]["D1"].iloc[0]

array([0.000e+00, 2.040e-02, 7.110e-02, 1.141e-01, 1.147e-01, 1.297e-01,
       1.772e-01, 1.856e-01, 1.369e-01, 3.200e-02, 1.340e-02, 3.800e-03,
       1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04,
       1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04])

: 

In [16]:
# Determine the maximum number of elements in any list
max_length = 100#max(df.applymap(len).max())

# Expand lists to have the same length
for col in df.columns:
    df[col] = df[col].apply(lambda x: x + [None] * (max_length - len(x)))

# Create new columns for each element in the lists
for i in range(max_length):
    df[f"A3_{i+1}"] = df['A3'].apply(lambda x: x[i])
    df[f"D1_{i+1}"] = df['D1'].apply(lambda x: x[i])
    df[f"D2_{i+1}"] = df['D2'].apply(lambda x: x[i])
    df[f"D3_{i+1}"] = df['D3'].apply(lambda x: x[i])
    df[f"D4_{i+1}"] = df['D4'].apply(lambda x: x[i])

# Drop the original columns with lists
df = df.drop(columns=['A3', 'D1', 'D2', 'D3', 'D4'])

TypeError: can only concatenate str (not "list") to str

In [52]:
#computes euclidean distance between 2 vectors
def euclidean_distance(vector1, vector2):
    euclidean = np.linalg.norm(np.asarray(vector1[2:9]) - np.asarray(vector2[2:9]))
    return euclidean

#computes cosine distance between 2 vectors
def cosine_distance(vector1, vector2):
    one_arr = np.asarray(vector1[2:9])
    two_arr = np.asarray(vector2[2:9])
    cosine = 1 - ((one_arr @ two_arr) / (np.linalg.norm(one_arr) * np.linalg.norm(two_arr)))
    return cosine

In [125]:
data = np.array([])
data = np.array([1.0,2,3,4,5,6,7])
for index, row in df.iterrows():
    # datarow = 
    data = np.stack((data,row[2:9]),axis=0)

ValueError: all input arrays must have the same shape

In [109]:
data.shape

(17325,)

In [128]:

df["class"]

0       AircraftBuoyant
1       AircraftBuoyant
2       AircraftBuoyant
3       AircraftBuoyant
4       AircraftBuoyant
             ...       
2470         WheelChair
2471         WheelChair
2472         WheelChair
2473         WheelChair
2474         WheelChair
Name: class, Length: 2475, dtype: object

In [184]:
npdata = df[df.columns[2:9]].to_numpy()

In [131]:
df[df.columns[2:9]].to_numpy().shape

(2475, 7)

In [ ]:
npdata.hstack(npdata, df[df.columns[9]].to_numpy().flatten)

In [157]:
df[df.columns[9]].to_numpy()[0].shape

(100,)

In [173]:
npcol = df[df.columns[9]].to_numpy()

In [174]:
np.array([a for a in npcol]).shape

(2475, 100)

In [180]:
npdata = np.hstack((npdata,np.array([a for a in npcol])))

In [185]:

for col in df.columns[9:]:
    npcol = df[col].to_numpy()
    npdata = np.hstack((npdata,np.array([a for a in npcol])))

In [285]:
npdata.shape

(2475, 507)

In [204]:
labels = df["class"]
labels.shape

(2475,)

In [233]:
scalar_features = df[df.columns[2:9]].to_numpy()

In [306]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Assuming 'labels' is your string label array
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Now, you can one-hot encode the integer labels
labels_one_hot = to_categorical(labels_encoded)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(npdata, labels_one_hot, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(scalar_features, labels_one_hot, test_size=0.2, random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [307]:
import tensorflow as tf


In [308]:
X_train.shape[1]

507

In [318]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation="softmax")
])


In [319]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [320]:
np.argmax(X_train)

258065

In [321]:
history = model.fit(X_train, y_train, epochs=60, batch_size=48, validation_data=(X_val, y_val))


Epoch 1/60
42/42 [==============================] - 3s 20ms/step - loss: 0.2487 - accuracy: 0.0237 - val_loss: 0.0885 - val_accuracy: 0.0242
Epoch 2/60
42/42 [==============================] - 0s 7ms/step - loss: 0.0776 - accuracy: 0.0561 - val_loss: 0.0764 - val_accuracy: 0.0363
Epoch 3/60
42/42 [==============================] - 0s 7ms/step - loss: 0.0725 - accuracy: 0.0828 - val_loss: 0.0729 - val_accuracy: 0.0565
Epoch 4/60
42/42 [==============================] - 0s 7ms/step - loss: 0.0693 - accuracy: 0.1086 - val_loss: 0.0693 - val_accuracy: 0.0927
Epoch 5/60
42/42 [==============================] - 0s 7ms/step - loss: 0.0662 - accuracy: 0.1258 - val_loss: 0.0669 - val_accuracy: 0.1048
Epoch 6/60
42/42 [==============================] - 0s 10ms/step - loss: 0.0634 - accuracy: 0.1697 - val_loss: 0.0641 - val_accuracy: 0.1250
Epoch 7/60
42/42 [==============================] - 0s 7ms/step - loss: 0.0613 - accuracy: 0.1737 - val_loss: 0.0630 - val_accuracy: 0.1411
Epoch 8/60
42/42 [

In [298]:
model.evaluate(X_test,y_test)

8/8 [==============================] - 0s 3ms/step - loss: 0.0482 - accuracy: 0.5263


[0.048181045800447464, 0.5263158082962036]

In [302]:
history = model.fit(X_train, y_train, epochs=90, batch_size=48, validation_data=(X_test, y_test))


Epoch 1/90
42/42 [==============================] - 2s 18ms/step - loss: 0.2648 - accuracy: 0.0263 - val_loss: 0.0883 - val_accuracy: 0.1012
Epoch 2/90
42/42 [==============================] - 0s 7ms/step - loss: 0.0854 - accuracy: 0.0687 - val_loss: 0.0794 - val_accuracy: 0.1498
Epoch 3/90
42/42 [==============================] - 0s 11ms/step - loss: 0.0769 - accuracy: 0.0975 - val_loss: 0.0733 - val_accuracy: 0.1619
Epoch 4/90
42/42 [==============================] - 0s 7ms/step - loss: 0.0707 - accuracy: 0.1313 - val_loss: 0.0685 - val_accuracy: 0.1943
Epoch 5/90
42/42 [==============================] - 0s 7ms/step - loss: 0.0663 - accuracy: 0.1631 - val_loss: 0.0645 - val_accuracy: 0.1984
Epoch 6/90
42/42 [==============================] - 0s 7ms/step - loss: 0.0634 - accuracy: 0.1586 - val_loss: 0.0627 - val_accuracy: 0.1619
Epoch 7/90
42/42 [==============================] - 0s 7ms/step - loss: 0.0609 - accuracy: 0.1753 - val_loss: 0.0609 - val_accuracy: 0.2227
Epoch 8/90
42/42 [

In [303]:
model.evaluate(X_test,y_test)

8/8 [==============================] - 0s 16ms/step - loss: 0.0537 - accuracy: 0.4170


[0.05372247099876404, 0.4170040488243103]

In [327]:
model.predict(np.expand_dims(X_test[1],axis=0))

1/1 [==============================] - 0s 180ms/step


array([[6.0881739e-05, 4.3477281e-04, 2.2154581e-02, 4.7904081e-12,
        3.2821693e-16, 3.6386155e-11, 1.4119927e-04, 2.4687927e-06,
        2.6710299e-05, 1.6515616e-02, 2.6358837e-05, 1.0057652e-03,
        2.7855140e-01, 6.1845974e-05, 1.2478083e-06, 1.4544706e-10,
        1.9738731e-09, 2.6238002e-05, 9.9166853e-10, 2.8255085e-08,
        3.5686286e-03, 1.0884575e-03, 1.3306445e-06, 3.4942225e-04,
        5.0180535e-07, 2.5484384e-10, 1.9093989e-04, 1.6568938e-02,
        1.7639359e-02, 6.4821370e-09, 1.8718160e-06, 1.8253517e-05,
        2.0747775e-07, 4.0796649e-01, 1.6899501e-04, 6.3728926e-06,
        1.0762562e-01, 1.1452646e-05, 3.5945442e-07, 4.3839838e-09,
        2.2869667e-08, 4.8971437e-14, 2.9637561e-09, 1.2207456e-04,
        1.3900997e-09, 1.2865679e-07, 5.1810851e-08, 1.9817989e-09,
        2.4666749e-02, 1.4335315e-09, 5.7258783e-04, 1.4539042e-11,
        1.7628191e-10, 1.4385607e-03, 2.6540773e-04, 6.8807121e-06,
        7.6398783e-07, 9.2081139e-03, 2.0491474e

In [218]:
model.weights[0][0]

<tf.Tensor: shape=(256,), dtype=float32, numpy=
array([ 0.08613849,  0.07744049,  0.07180903,  0.02806914,  0.10114263,
       -0.02230318, -0.1398607 ,  0.05064769,  0.00559836,  0.00803112,
       -0.08017083,  0.02782259,  0.09622506,  0.18158902,  0.01804393,
       -0.06855448, -0.01076055,  0.00230659, -0.04053889, -0.10038995,
       -0.03882802, -0.04569347, -0.1651528 , -0.11293713,  0.03648277,
       -0.11257657, -0.05594034,  0.03915012, -0.07056753, -0.07857022,
       -0.0224992 , -0.05083421, -0.09854107, -0.18424377, -0.01433188,
       -0.08216736,  0.12628052,  0.07744896,  0.020603  ,  0.04129937,
        0.02011634,  0.07948047, -0.09051403,  0.11207553, -0.04068328,
       -0.03597231,  0.11256795,  0.08580583, -0.0542618 , -0.00412015,
       -0.09311806,  0.08624953,  0.03870631,  0.03619569, -0.03944309,
        0.09944654,  0.11194292,  0.08082122,  0.00340634, -0.07702353,
        0.06261089,  0.00846741, -0.08012781, -0.01725913,  0.0533587 ,
       -0.031886

In [328]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_55 (Dense)            (None, 256)               130048    
                                                                 
 dense_56 (Dense)            (None, 512)               131584    
                                                                 
 dense_57 (Dense)            (None, 128)               65664     
                                                                 
 dense_58 (Dense)            (None, 69)                8901      
                                                                 
Total params: 336,197
Trainable params: 336,197
Non-trainable params: 0
_________________________________________________________________
